In [13]:
from dask.distributed import Client, SSHCluster
import os
import dask.bag as db
import dask.dataframe as dd
import json

In [17]:
client.close()

# Spin up Cluster and client

In [16]:
client = Client(n_workers=16,            # 16 processi separati
    threads_per_worker=1)  # Avvia un cluster locale con tutti i core disponibili
client


/home/gabriele/miniconda3/envs/mapd/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42495 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:42495/status,
Dashboard: http://127.0.0.1:42495/status,Workers: 16
Total threads: 16,Total memory: 30.85 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35105,Workers: 0
Dashboard: http://127.0.0.1:42495/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:45099,Total threads: 1
Dashboard: http://127.0.0.1:39311/status,Memory: 1.93 GiB
Nanny: tcp://127.0.0.1:44107,


In [10]:
bag = db.read_text("data/document_parses/pdf_json/*.json").map(json.loads)


In [12]:
def validate(doc):
    issues = []
    if not doc.get("paper_id"):
        issues.append("missing paper_id")
    if not isinstance(doc.get("metadata", {}).get("authors", []), list):
        issues.append("malformed authors")
    if not doc.get("metadata", {}).get("title"):
        issues.append("missing title")
    return {"paper_id": doc.get("paper_id", "UNKNOWN"), "issues": issues}

results = bag.map(validate)


df = results.to_dataframe()
df_clean = df[df["issues"].map(lambda x: len(x) > 0)]  # Solo problematici
df_clean.compute().to_csv("malformed_files.csv", index=False)


/home/gabriele/miniconda3/envs/mapd/lib/python3.13/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 174.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2025-06-28 18:13:20,814 - distributed.worker - ERROR - Compute Failed
Key:       bag-from-delayed-file_to_blocks-list-loads-take-validate-finalize-hlgfinalizecompute-274cc4a5607c42999ad3c7767a9101--4b2
State:     executing
Task:  <Task 'bag-from-delayed-file_to_blocks-list-loads-take-validate-finalize-hlgfinalizecompute-274cc4a5607c42999ad3c7767a9101--4b2' _execute_subgraph(...)>
Exception: "JSONDecodeError('Expecting property name enclosed in double quotes: line 2 column 1 (char 2)')"
Traceback: '  File "/home/gabriele/miniconda3/envs/mapd/lib/python3.13/site-pa

JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)

# Code

In [56]:
import dask.bag as db
import json


bag = db.read_text("/home/gabriele/Analysis-of-covid-19-papers-mapd-b-project/data/document_parses/pmc_json/*.json").map(json.loads)

KeyboardInterrupt: 

In [ ]:
bag.take(1)[0]

In [ ]:
def validate_paper(record):
    issues = []

    # Controllo presenza e tipo dei campi principali
    if not record.get("paper_id"):
        issues.append("Missing paper_id")

    metadata = record.get("metadata", {})
    if not isinstance(metadata, dict):
        issues.append("Malformed metadata")
    else:
        if not metadata.get("title"):
            issues.append("Missing metadata.title")
        if not isinstance(metadata.get("authors", []), list):
            issues.append("metadata.authors is not a list")

    abstract = record.get("abstract", [])
    if not isinstance(abstract, list):
        issues.append("abstract is not a list")

    return {
        "paper_id": record.get("paper_id", "UNKNOWN"),
        "issues": issues
    }


In [ ]:
validated = bag.map(validate_paper)

problems = validated.filter(lambda x: len(x["issues"]) > 0)
import pandas as pd

df = problems.compute()
pd.DataFrame(df).to_csv("malformed_summary.csv", index=False)


In [ ]:
def read__json(path):
    with os.scandir(path) as fil:
        if fil.endswith('.json'):

# CLOSE CLUSTER AND CLIENT WHEN DONE
I've found the shutdown thing, I don't know if it's the holy way

In [66]:
Client.close()  # closes the client (cluster is still up)
Cluster.close()
# client.shutdown()  # shutdowns cluster and client
# client.restart()  # restarts everything

TypeError: Client.close() missing 1 required positional argument: 'self'